In [1]:
## Pdf reader
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader(r'D:\AI\NLP\RAG_QA\PDFs\attention_is_all_you_need.pdf')
docs=loader.load()

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
# documents[:5]

In [6]:
## Vector Embedding And Vector Store
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
embeddings = (
    OllamaEmbeddings()
)
print(embeddings)
db = Chroma.from_documents(documents,embeddings)

base_url='http://localhost:11434' model='llama2' embed_instruction='passage: ' query_instruction='query: ' mirostat=None mirostat_eta=None mirostat_tau=None num_ctx=None num_gpu=None num_thread=None repeat_last_n=None repeat_penalty=None temperature=None stop=None tfs_z=None top_k=None top_p=None show_progress=False headers=None model_kwargs=None


In [7]:
query = "Who are the authors of attention is all you need?"
retireved_results=db.similarity_search(query)
print(retireved_results[0].page_content)

References
[1]Jimmy Lei Ba, Jamie Ryan Kiros, and Geoffrey E Hinton. Layer normalization. arXiv preprint
arXiv:1607.06450 , 2016.
[2]Dzmitry Bahdanau, Kyunghyun Cho, and Yoshua Bengio. Neural machine translation by jointly
learning to align and translate. CoRR , abs/1409.0473, 2014.
[3]Denny Britz, Anna Goldie, Minh-Thang Luong, and Quoc V . Le. Massive exploration of neural
machine translation architectures. CoRR , abs/1703.03906, 2017.
[4]Jianpeng Cheng, Li Dong, and Mirella Lapata. Long short-term memory-networks for machine
reading. arXiv preprint arXiv:1601.06733 , 2016.
[5]Kyunghyun Cho, Bart van Merrienboer, Caglar Gulcehre, Fethi Bougares, Holger Schwenk,
and Yoshua Bengio. Learning phrase representations using rnn encoder-decoder for statistical
machine translation. CoRR , abs/1406.1078, 2014.
[6]Francois Chollet. Xception: Deep learning with depthwise separable convolutions. arXiv
preprint arXiv:1610.02357 , 2016.


In [8]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="llama2")
llm

Ollama()

In [15]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Don't make any mistakes. 
<context>
{context}
</context>
Question: {input}""")

In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [17]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001AB7AAA4A30>)

In [18]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [19]:
response=retrieval_chain.invoke({"input":"Who is the author of this book"})

In [ ]:
response['answer']

'Based on the provided context, the author of the book is most likely to be either of the following:\n\n1. Vaswani et al.: The authors of the paper "Attention Is All You Need" which introduced the Transformer model and discussed its applications in sequence-to-sequence tasks.\n2. BERT team: The authors of the BERT (Bidirectional Encoder Representations from Transformers) model, which is a variant of the Transformer architecture that has been pre-trained on a large corpus of text data and has achieved state-of-the-art results in various NLP tasks.\n\nReasoning:\nThe passage mentions the Transformer model and its applications in sequence-to-sequence tasks, which are key features of the BERT model as well. The authors of the paper "Attention Is All You Need" are likely to be the ones who developed the Transformer architecture and its applications in NLP tasks. However, since the passage also mentions the BERT team, it is possible that they are also the authors of the book.'